# <span style="color:red">Open</span>: goroutines should be canceled when the main routine ends
- [bug](https://github.com/yunabe/lgo/issues/6)

In [ ]:
i := 0
go func() {
    for i = 0;; i++ {}
}()

# <span style="color:red">Open</span>: The entire process terminated when a go routine panics
- [bug](https://github.com/yunabe/lgo/issues/5)

In [ ]:
go func() {
    panic("die!")
}()

# <span style="color:green">Fixed</span>: interface is not working
If you invoke a method through an interface in lgo, it crashes with `runtime error: invalid memory address or nil pointer dereference` ([bug](https://github.com/yunabe/lgo/issues/3)).

In [1]:
import (
    "fmt"
)

type Hello interface {
    SayHello()
}

type person struct {
    name string
}

func (p *person) SayHello() {
    fmt.Printf("Hello, I'm %s.\n", p.name)
}

p := person{"yunabe"}
fmt.Println("---- 1 ----")
p.SayHello()

var h Hello = &p
fmt.Println("---- 2 ----")
h.SayHello()

---- 1 ----
Hello, I'm yunabe.
---- 2 ----
Hello, I'm yunabe.


# <span style="color:green">Fixed</span>: GC crashes with a fatal error
The following crashes with `fatal error: found bad pointer in Go heap (incorrect use of unsafe or cgo?)` message ([bug](https://github.com/yunabe/lgo/issues/2)).

In [1]:
import (
    "fmt"
    "log"
    "runtime"
    "runtime/debug"
)

type MyData struct {
    b []byte
}

func (m *MyData) Size() int {
    return len(m.b)
}

func NewMyData() *MyData {
    return &MyData{
        b: make([]byte, 10 * (1 << 20)),
    }
}

var l []*MyData
for i := 0; i < 100; i++ {
    d := NewMyData()
    l = append(l, d)
}
l = nil
debug.FreeOSMemory()
runtime.GC()